In [ ]:
import os
import time
import numpy as np
import rasterio
from tqdm import tqdm
from pathlib import Path
import glob

In [ ]:
def process_raster(input_file, out_path):
    """
    Process a single raster file through the decision tree classification.
    
    Args:
        input_file (str): Path to input raster file
        out_path (str): Directory to save output files
    """
    # Timer
    start_time = time.time()
    
    # Extract the base name of the input file without extension
    input_base_name = Path(input_file).stem
    
    print(f"Processing file: {input_file}")
    
    # Check to see if outpath exists
    os.makedirs(out_path, exist_ok=True)
    
    with rasterio.open(input_file) as src:
        # Get dimensions from the source
        width = src.width
        height = src.height
        
        # Use a window that covers the entire raster
        window = rasterio.windows.Window(col_off=0, row_off=0, width=width, height=height)
        data = src.read(1, window=window)
        profile = src.profile  # defines profile
        print("Raster data read successfully.")
        
    # Apply the reclassification function to each pixel
    reclassified_data = np.where(data > 1, 300,
                                np.where(data < -1, 100,
                                        np.where((data >= -1) & (data <= 1), 0, 200)))
    print("Reclass Complete")
    
    # Create masks for each reclassification category in order to export individually
    mask_300 = np.where(reclassified_data == 300, 300, 0).astype(np.uint8)
    mask_200 = np.where(reclassified_data == 200, 200, 0).astype(np.uint8)
    mask_100 = np.where(reclassified_data == 100, 100, 0).astype(np.uint8)
    print("Masking Complete")
    
    # Get CRS from source
    crs = profile.get('crs')
    
    # Save each reclassification to separate files (and provide progress bar)
    output_files = {
        f"{input_base_name}_pn_300.tif": mask_300,
        f"{input_base_name}_pn_200.tif": mask_200,
        f"{input_base_name}_pn_100.tif": mask_100
    }
    
    for filename, reclass_data in tqdm(output_files.items(), desc="Processing Reclass Files"):
        out_file_path = os.path.join(out_path, filename)
        # Check to see if file exists before saving file
        if os.path.exists(out_file_path):
            os.remove(out_file_path)
        # Save file
        with rasterio.open(out_file_path, 'w', driver='GTiff', height=reclass_data.shape[0],
                          width=reclass_data.shape[1], count=1, dtype=reclass_data.dtype,
                          crs=crs, transform=profile['transform'], compress='lzw') as dst:
            dst.write(reclass_data, 1)
            print(f"File saved: {out_file_path}")
    print(f"Reclassified patch number rasters saved for {input_base_name}.")
    
    # Time
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total Elapsed Time: {total_time:.2f} seconds")

def batch_process_rasters(input_pattern, out_path):
    """
    Process multiple raster files using a glob pattern.
    
    Args:
        input_pattern (str): Glob pattern to find input files
        out_path (str): Directory to save output files
    """
    # Start timer
    start_time = time.time()
    
    # Create output directory if it doesn't exist
    os.makedirs(out_path, exist_ok=True)
    
    # Get list of input files
    input_files = glob.glob(input_pattern)
    
    if not input_files:
        print(f"No files found matching pattern: {input_pattern}")
        return
    
    print(f"Found {len(input_files)} files to process")
    
    # Process each file
    for input_file in input_files:
        process_raster(input_file, out_path)
    
    # End timer and print total time
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total Batch Processing Time: {total_time:.2f} seconds")

def main():
    # Example usage
    input_pattern = "E:/mosaic_reprojected/Overlay_calculation/*.tif"
    out_path = "E:/mosaic_reprojected/DecisionTree"
    
    # To process multiple files
    batch_process_rasters(input_pattern, out_path)

if __name__ == "__main__":
    main()
